In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
all_files = [
    './ql/Clean-Data-01.txt',
    './ql/Clean-Data-02.txt',
    './ql/Clean-Data-03.txt',
    './ql/Clean-Data-04.txt',
    './ql/Clean-Data-05.txt'
]

df = pd.concat((pd.read_csv(f, sep='\t') for f in all_files))

In [3]:
df

,AnonID,Query,QueryTime
0,142,merit release appearance,2006-04-22 23:51:18
1,217,lottery,2006-03-01 11:58:51
2,217,lottery,2006-03-27 14:10:38
3,217,vietnam,2006-05-22 17:43:42
4,217,vietnam,2006-05-22 18:03:24
5,1268,gall stones,2006-05-11 02:12:51
6,1268,gallstones,2006-05-11 02:13:02
7,1326,files,2006-03-01 17:36:08
8,1326,pop up adds,2006-03-15 20:07:38
9,1326,pop up adds,2006-03-15 20:08:29


In [4]:
# converts a string to a set of words
def make_set(x):
    return set(str(x).split())

# split string, put in set, and then convert again to string
def split_join(x):
    x = set(x.split())
    return ' '.join(x)

In [5]:
# convert QueryTime string to POSIX timestamp
df['QueryTime'] = pd.to_datetime(df['QueryTime'])
df['q_time'] = df['QueryTime'].map(lambda x: x.timestamp())

In [6]:
# convert query strings to sets of words
df['q_split'] = df['Query'].map(lambda x: make_set(x))

In [7]:
# get lengths of queries
df['q_length'] = df['q_split'].map(lambda x: len(x))

In [8]:
# convert set back to string
df['q_split_join'] = df['q_split'].map(lambda x: ' '.join(x))

In [9]:
df

,AnonID,Query,QueryTime,q_time,q_split,q_length,q_split_join
0,142,merit release appearance,2006-04-22 23:51:18,1.145750e+09,"{release, merit, appearance}",3,release merit appearance
1,217,lottery,2006-03-01 11:58:51,1.141214e+09,{lottery},1,lottery
2,217,lottery,2006-03-27 14:10:38,1.143469e+09,{lottery},1,lottery
3,217,vietnam,2006-05-22 17:43:42,1.148320e+09,{vietnam},1,vietnam
4,217,vietnam,2006-05-22 18:03:24,1.148321e+09,{vietnam},1,vietnam
5,1268,gall stones,2006-05-11 02:12:51,1.147314e+09,"{stones, gall}",2,stones gall
6,1268,gallstones,2006-05-11 02:13:02,1.147314e+09,{gallstones},1,gallstones
7,1326,files,2006-03-01 17:36:08,1.141235e+09,{files},1,files
8,1326,pop up adds,2006-03-15 20:07:38,1.142453e+09,"{pop, adds, up}",3,pop adds up
9,1326,pop up adds,2006-03-15 20:08:29,1.142453e+09,"{pop, adds, up}",3,pop adds up


### Candidate queries

In [10]:
# gets candidate queries and the sessions where those candidate queries occur
def get_sessions(q):
    # find indeces where the user's query is part of a query in the query log, and apply to dataframe
    df_criterion = df['q_split'].map(lambda x: q.issubset(x) and len(x) == (len(q)+1))
    qdf = df[df_criterion]
    # return searches in same session as the query, and candidate queries
    return df[df.AnonID.isin(qdf.AnonID.values)], qdf

### Freq(CQ)
frequency of CQ in QL / max freq of any query in QL

In [11]:
def calc_frequencies():
    df_gb = df.groupby(['Query'])
    df_freq = pd.DataFrame(df_gb.count()['q_length'])
    # TODO: do log?
    max_freq = df_freq['q_length'].sort_values(ascending=False)[0]
    df_freq['freq'] = df_freq['q_length'].map(lambda x: x/max_freq)
    return df_freq['freq']

In [12]:
def freq(CQ):
    return frequencies.loc[CQ]

### Mod(CQ,q')
num sessions q' is modified to CQ / num sessions q' appears in QL

In [13]:
# gets the indeces of the query in the sessions
def get_qsessions(q):
    s_criterion = sessions['q_split'].map(lambda x: x == q)
    q_sessions = sessions[s_criterion]
    return q_sessions

In [14]:
def mod(CQ, q):
    
    # split candidate string, put in set, and then convert to string
    CQ = split_join(CQ)
    
    # get the indeces of the query in the sessions
    q_sessions = get_qsessions(q)
    q_index = q_sessions.index
    
    # num sessions q' is modified to CQ
    # TODO: per-session instead
    mod_count = 0
    for i in q_index:
#         if sessions['q_split'].loc[i+1]  == CQ:
        if sessions['q_split_join'].loc[i+1]  == CQ:
            mod_count += 1

    # num sessions q' appears in QL
#     s_gb = q_sessions[['Query', 'AnonID']].groupby(['Query']).nunique()
#     s_gb = df[['Query', 'AnonID']].groupby(['Query']).nunique()
#     q_count = s_gb.count()[0]
    q_count = q_num_count
    
    return mod_count / q_count

### Time(CQ,q')
min diff between times of q' and CQ in sessions / length of longest session in QL

In [15]:
# get length of longest session in QL
def get_max_time():
    session_lengths = df.groupby('AnonID')['q_time'].agg(np.ptp)
    return session_lengths.sort_values(ascending=False).iloc[0]

In [16]:
def time(CQ, q):
    CQ = split_join(CQ)
    q_sessions = get_qsessions(q)
    q_index = q_sessions.index
    min_time = 0 # in seconds
    for i in q_index:
        if sessions['q_split_join'].loc[i+1]  == CQ:
            time_range = sessions['q_time'].loc[i+1] - sessions['q_time'].loc[i]
            if time_range < min_time or min_time == 0:
                min_time = time_range
    return min_time

### Score(CQ,q')
(Freq(CQ) + Mod(CQ,q') + Time(CQ,q')) / (1 - Min{Freq(CQ,q'), Mod(CQ,q'), Time(CQ,q'))

In [17]:
def score(CQ, q):
    fr = freq(CQ)
    mo = mod(CQ, q)
    ti = time(CQ, q)
    return (fr + mo + ti) / (1 - min(fr, mo, ti))

### Get query suggestions

In [18]:
query = ['make', 'money']
query = set(query)

In [19]:
print('calculating all frequencies...')
frequencies = calc_frequencies()

calculating all frequencies...


In [20]:
print('getting max session length...')
max_session_length = get_max_time()

getting max session length...


/home/stacyblack/.local/bin/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [21]:
print('getting sessions and candidates...')
sessions, candidates = get_sessions(query)

getting sessions and candidates...


In [22]:
print('getting num sessions where q appears...')
s_gb = df[['Query', 'AnonID']].groupby(['Query']).nunique()
q_num_count = s_gb.count()[0]

getting num sessions where q appears...


In [23]:
candidates

,AnonID,Query,QueryTime,q_time,q_split,q_length,q_split_join
50523,543587,make money surfing,2006-04-21 16:03:10,1.145635e+09,"{surfing, money, make}",3,surfing money make
150956,1714713,make money fast,2006-03-30 16:13:32,1.143735e+09,"{fast, money, make}",3,fast money make
166439,1863035,make easy money,2006-03-01 01:58:38,1.141178e+09,"{easy, money, make}",3,easy money make
166460,1863035,make money fast,2006-03-11 23:24:06,1.142119e+09,"{fast, money, make}",3,fast money make
269525,2983651,make money ideas,2006-03-10 10:29:50,1.141987e+09,"{ideas, money, make}",3,ideas money make
285084,3151268,make money now,2006-03-26 15:54:41,1.143388e+09,"{now, money, make}",3,now money make
434301,5673422,make money overnight,2006-05-08 20:00:51,1.147118e+09,"{overnight, money, make}",3,overnight money make
468840,6458100,make fast money,2006-04-06 15:09:54,1.144336e+09,"{fast, money, make}",3,fast money make
527702,8112331,make money fast,2006-05-11 18:41:27,1.147373e+09,"{fast, money, make}",3,fast money make
602489,10792134,make money fast,2006-04-26 18:47:34,1.146077e+09,"{fast, money, make}",3,fast money make


In [24]:
score('make money fast', query)

0.00023901430500615462

In [27]:
# get top 5 query suggestions
def get_top_5(cand, q):
    
    q = set(q)
    
#     print('getting sessions and candidates...')
    # searches in same session as the query, and candidate queries
#     sessions, candidates = get_sessions(q)
    
#     print('calculating all frequencies...')
#     frequencies = calc_freqencies()
#     max_session_length = get_max_time()
    
    cand['score'] = cand['Query'].map(lambda x: score(x, q))
    cand = cand.sort_values(by='score', ascending=False)
    # drop query suggestions that are duplicates (just different arrangement of words)
    top_10 = cand.drop_duplicates(subset='q_split_join')[:5]
    return top_10['Query'].values

In [28]:
get_top_5(candidates, query)

/home/stacyblack/.local/bin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


array(['make easy money', 'make money fast', 'make money survey',
       'make money overnight', 'make money now'], dtype=object)